# Which network should I use?

As of the release of OpenPathSampling 1.0, even people who are familiar with path sampling are pretty new to the various multiple state approaches --- so don't feel bad if that seems a bit overwhelming.

Here's a list of likely situations you might find yourself in, with advice on which method to use. In 2-state situations, both the `MSTISNetwork` and `MISTISNetwork` will give the same behavior, so it doesn't matter which you use. We recommend using the `MSTISNetwork`, because its syntax is a bit easier. However, below we'll show you how to do both.

In addition to the choice of network, which defines the interfaces in your system, we also discuss the single replica approach, which is an alternative method

In [1]:
import openpathsampling as paths

## Two states, only one direction (A->B without B->A)

In this case, your best tool is probably the `MISTISNetwork`. This amounts to just having one interface set.

## Two states, both directions (A->B and B->A)

In this case, you can use either the `MISTISNetwork` or the `MSTISNetwork`. Both will create the same ensembles and the same default move scheme. However, we think the `MSTISNetwork` notation is a little easier. It looks like this:

In [2]:
paths.MSTISNetwork([
    (stateA, interfacesA, orderparameterA),
    (stateB, interfacesB, orderparameterB)        
])

NameError: name 'stateA' is not defined

Each state is associated with a set of interfaces and an order parameter.

## Multiple States

Once we get into the world of multiple states, it gets more complicated. This is partly inevitable, but partly also because multiple state transition sampling methods are relatively recent, dating back to the work of Rogal and Bolhuis in 2008.

The main approaches to multiple states in TIS are Multiple State TIS (MSTIS) and Multiple Interface Set TIS (MISTIS). Both are fully supported by OpenPathSampling. In MSTIS, there is one order parameter associated with each state. In MISTIS, there is one order parameter associated with each transitions.

This means that, depending on the nature of the network of transitions to be studied, one or the other of these approaches might be better.

The advantages of the MISTIS approach are:

* it's easier to study networks where two transitions from a given state are very different
* you can selectively increase the sampling of rarer transitions
* you can easily study a subset of the total transition network

The advantages of the MSTIS approach are:

* you can use adaptive methods to discover new states
* it requires far fewer interfaces
* it is easier to set up
* the flux can be calculated from the minus interface (this is hypothetically possible with MISTIS, but not yet implemented in OPS)

As you might guess from these lists, if your system is easy to study, it might be better to use an MSTIS network. However, the MISTIS network provides more flexibility.

## Customizing behavior within a network

The default network behavior should work for most situations. However, if you have particular needs, it is easy to tweak your simulation to meet those needs before you run it.

However, to do this you need to learn a little more about the underlying nature of how networks are implemented. This is a brief introduction to that.

First a network can be described as consisting of several transitions. Each transition is a one-directional link between two "states" (although sometimes several stable states are lumped into one).

There are two sides to each network: there's the sampling network, which describes how you sample the transitions, and the analysis network, which describes 

## Single Replica Transition Interface Sampling

One of the challenges of TIS is that it doesn't naturally parallelize well. Of course, each trajectory can be parallelized as much as possible. However, one could easily do shooting moves on *all* ensembles at the same time, and parallelize further by putting each trajectory on its own set of nodes. The problem with this is that not all trajectories will take the same amount of time. The path length for each ensemble will be different, and what's more, within each ensemble there is a wide range of path lengths. So if you run shooting moves in all ensembles at the same time, some of your nodes will finish long before the others.

One of the best approaches which has been developed to fix this problem is Single Replica Transition Interface Sampling (SRTIS), which samples the entire space of ensembles and paths using a single replica. This turns parallelization of RETIS into an embarassingly parallel problem: you can have as many replicas as you would like spread across as many machines as you would like.